In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, desc, lower
import re
EXPRESSIONS_SIZE = 3

from utils import get_session

spark = get_session()

df = spark.read.csv('hdfs:///tweets/debate-tweets-001.tsv', sep='\t').select('_c1', '_c7', '_c8')
df.show()

Setting default log level to "

22/12/08 22:44:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
+--------------------+--------------------+----------+
|                 _c1|                 _c7|       _c8|
+--------------------+--------------------+----------+
|@anacddd verdade,...|Wed Oct 15 14:31:...|2014-10-15|
|              Que ñ*|Wed Oct 15 14:31:...|2014-10-15|
| Vou quebrar a Bruna|Wed Oct 15 14:31:...|2014-10-15|
|agora vou p segun...|Wed Oct 15 14:31:...|2014-10-15|
|Me sinto tão bem ...|Wed Oct 15 14:31:...|2014-10-15|
|Eu estou aqui, de...|Wed Oct 15 14:31:...|2014-10-15|
|Quando vai embora...|Wed Oct 15 14:31:...|2014-10-15|
|@paynecaralhudo k...|Wed Oct 15 14:31:...|2014-10-15|
|Conceição da Barr...|Wed Oct 15 14:31:...|2014-10-15|
| @Maniavato te amo ♥|Wed Oct 15 14:31:...|2014-10-15|
|Alg me curtindo rs ♡|Wed Oct 15 14:31:...|2014-10-15|
|@MiiluAA No, porq...|Wed Oct 15 14:31:...|2014-10-15|
|#EMABiggestFansJu...|Wed Oct 15

In [2]:
#Filter by tweets that contains 'Dilma' and match with other tweets that have the same sentences
from pyspark.ml.feature import NGram

df2 = df.select(split(lower(df._c1),' ').alias('words')).na.drop()
ngram_gen = NGram(n=EXPRESSIONS_SIZE, inputCol="words", outputCol="phrases")
processed_df = ngram_gen.transform(df2)

df3 = processed_df.select(explode(processed_df.phrases)).withColumnRenamed('col','sentences')
df4 = df3.filter(df3.sentences.contains('Dilma') | df3.sentences.contains('dilma'))
df5 = df4.groupBy(df4.sentences).count().orderBy(desc('count'))
df5.show()

+------------------+-----+
|         sentences|count|
+------------------+-----+
|       que a dilma|  758|
|     cara da dilma|  452|
|     dilma e aécio|  241|
|       a dilma não|  228|
|    votar na dilma|  214|
|       com a dilma|  203|
|         a dilma é|  191|
|        se a dilma|  187|
|        a dilma ta|  185|
|         a dilma e|  183|
|         e a dilma|  180|
|       a dilma vai|  172|
|        da dilma e|  158|
|        da dilma é|  149|
|         dilma e o|  137|
|dilma passando mal|  135|
|         q a dilma|  129|
|     entre dilma e|  118|
|        a dilma tá|  114|
|     aécio e dilma|  114|
+------------------+-----+
only showing top 20 rows



In [3]:
spark.stop()